# 🐝 Join the GLTCH Hive

**Contribute your GPU to train AI together!**

This notebook connects to the GLTCH distributed training network. Your GPU will help train open-source language models alongside other contributors.

---

## Quick Start

1. **Runtime → Change runtime type → T4 GPU** (or better)
2. **Run all cells** (Ctrl+F9 or Runtime → Run all)
3. Watch your GPU contribute to training! 🚀

## Step 1: Install Dependencies

In [ ]:
!pip install torch websockets requests -q
print("✅ Dependencies installed!")

## Step 2: Check GPU

In [ ]:
import torch

if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    print(f"🎮 GPU detected: {gpu_name}")
    print(f"   VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("⚠️ No GPU detected. Go to Runtime → Change runtime type → T4 GPU")

## Step 3: Download Peer Script

In [ ]:
!wget -q https://raw.githubusercontent.com/cyberdreadx/gltch-llm/main/hive/peer.py -O peer.py
print("✅ Peer script downloaded!")

## Step 4: Configuration

Customize your peer settings below:

In [ ]:
# ═══════════════════════════════════════════════════════
# CONFIGURATION - Customize these settings!
# ═══════════════════════════════════════════════════════

# Your peer's display name (shown on dashboard)
PEER_NAME = "colab-contributor"

# Model size: '1m' (fastest), '2.7m', '10m', '25m', '50m' (largest)
MODEL_SIZE = "10m"

# Server connection (public hive)
SERVER_URL = "ws://76.13.121.10:8765"
AUTH_KEY = "PUBLIC_KEY"

# ═══════════════════════════════════════════════════════

print(f"📋 Configuration:")
print(f"   Name: {PEER_NAME}")
print(f"   Model: GLTCH-{MODEL_SIZE.upper()}")
print(f"   Server: {SERVER_URL}")

## Step 5: Join the Hive! 🐝

Run this cell to start training. Your GPU will:
- Connect to the coordinator
- Download training data
- Train and share gradients with other peers

**Watch the live dashboard:** https://hive.gltch.app

In [ ]:
!python peer.py --server {SERVER_URL} --key {AUTH_KEY} --name {PEER_NAME} --size {MODEL_SIZE}

---

## 🧪 Test the Model

After training, you can chat with the model! Run the cells below to generate text.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Head(nn.Module):
    def __init__(self, head_size, n_embd, block_size, dropout):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
        k, q = self.key(x), self.query(x)
        wei = q @ k.transpose(-2,-1) * C**-0.5
        wei = wei.masked_fill(self.tril[:T,:T] == 0, float('-inf'))
        wei = self.dropout(F.softmax(wei, dim=-1))
        return wei @ self.value(x)

class MultiHeadAttention(nn.Module):
    def __init__(self, n_heads, head_size, n_embd, block_size, dropout):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size, n_embd, block_size, dropout) for _ in range(n_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)
    def forward(self, x):
        return self.dropout(self.proj(torch.cat([h(x) for h in self.heads], dim=-1)))

class FeedForward(nn.Module):
    def __init__(self, n_embd, dropout):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(n_embd, 4*n_embd), nn.ReLU(), nn.Linear(4*n_embd, n_embd), nn.Dropout(dropout))
    def forward(self, x): return self.net(x)

class Block(nn.Module):
    def __init__(self, n_embd, n_head, block_size, dropout):
        super().__init__()
        self.sa = MultiHeadAttention(n_head, n_embd//n_head, n_embd, block_size, dropout)
        self.ffwd = FeedForward(n_embd, dropout)
        self.ln1, self.ln2 = nn.LayerNorm(n_embd), nn.LayerNorm(n_embd)
    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        return x + self.ffwd(self.ln2(x))

class GLTCH(nn.Module):
    def __init__(self, vocab_size, n_embd=384, n_head=6, n_layer=6, block_size=256, dropout=0.2):
        super().__init__()
        self.block_size = block_size
        self.tok_emb = nn.Embedding(vocab_size, n_embd)
        self.pos_emb = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head, block_size, dropout) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx): 
        B, T = idx.shape
        x = self.tok_emb(idx) + self.pos_emb(torch.arange(T, device=idx.device))
        return self.lm_head(self.ln_f(self.blocks(x)))

    def generate(self, idx, max_tokens):
        for _ in range(max_tokens):
            logits = self(idx[:, -self.block_size:])[:, -1, :]
            idx = torch.cat((idx, torch.multinomial(F.softmax(logits, dim=-1), 1)), dim=1)
        return idx

print('✅ Model classes loaded!')

In [ ]:
#@title 💬 Chat with GLTCH { run: "auto" }
prompt = "KING: To be or not" #@param {type:"string"}
max_tokens = 200 #@param {type:"slider", min:50, max:500, step:50}

device = 'cuda' if torch.cuda.is_available() else 'cpu'
checkpoint = torch.load('gltch_model.pt', map_location=device)

stoi = checkpoint.get('stoi', {chr(i): i for i in range(128)})
itos = {v: k for k, v in stoi.items()}

model = GLTCH(len(stoi)).to(device)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

encode = lambda s: [stoi.get(c, 0) for c in s]
decode = lambda l: ''.join([itos.get(i, '?') for i in l])

with torch.no_grad():
    context = torch.tensor([encode(prompt)], device=device)
    output = decode(model.generate(context, max_tokens)[0].tolist())

print('═' * 50)
print('🤖 GLTCH says:')
print('═' * 50)
print(output)

---

## 📊 Training Stats

| Model Size | Parameters | VRAM | Speed |
|------------|------------|------|-------|
| 1m | ~1M | <1GB | Very Fast |
| 2.7m | 2.7M | ~1GB | Fast |
| 10m | ~10M | ~2GB | Normal |
| 25m | ~25M | ~4GB | Slow |
| 50m | ~50M | ~8GB | Slowest |

---

## 🔗 Links

- **Dashboard:** https://hive.gltch.app
- **GitHub:** https://github.com/cyberdreadx/gltch-llm
- **Created by:** cyberdreadx